In [ ]:
# prompt: connect google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
import pandas as pd

# Load the CSV file
file_path = '/content/drive/MyDrive/NLP MINI PROJECT/data_to_be_cleansed.csv'
df = pd.read_csv(file_path)

# Display the first few rows
df.head()

,Unnamed: 0,text,title,target
0,0,Welcome to /r/depression's check-in post - a p...,"Regular check-in post, with information about ...",1
1,1,We understand that most people who reply immed...,Our most-broken and least-understood rules is ...,1
2,2,Anyone else just miss physical touch? I crave ...,"I haven’t been touched, or even hugged, in so ...",1
3,3,I’m just so ashamed. Everyone and everything f...,Being Depressed is Embarrassing,1
4,4,I really need a friend. I don't even have a si...,I'm desperate for a friend and to feel loved b...,1


In [ ]:
df1 = df.drop(df.columns[0], axis=1)

In [ ]:
df1 = df1.drop(df.columns[2], axis=1)

In [ ]:
df1.head()

,text,target
0,Welcome to /r/depression's check-in post - a p...,1
1,We understand that most people who reply immed...,1
2,Anyone else just miss physical touch? I crave ...,1
3,I’m just so ashamed. Everyone and everything f...,1
4,I really need a friend. I don't even have a si...,1


In [ ]:
# Load the CSV file
file_path = "C:\Users\kochs\OneDrive\Desktop\NLP Dataset\depression_dataset_reddit_cleaned.csv"
df2 = pd.read_csv(file_path)

In [ ]:
df2.head()

,clean_text,is_depression
0,we understand that most people who reply immed...,1
1,welcome to r depression s check in post a plac...,1
2,anyone else instead of sleeping more when depr...,1
3,i ve kind of stuffed around a lot in my life d...,1
4,sleep is my greatest and most comforting escap...,1


In [ ]:
df2.shape

(7731, 2)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7731 entries, 0 to 7730
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_text     7731 non-null   object
 1   is_depression  7731 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 120.9+ KB


In [ ]:
df2['is_depression'].value_counts()

0    3900
1    3831
Name: is_depression, dtype: int64

In [ ]:
# Drop rows where 'is_depression' is equal to 1
df2 = df2.drop(df2[df2['is_depression'] == 1].index)
df2.head()

,clean_text,is_depression
3831,switchfoot http twitpic com y zl awww that s a...,0
3832,is upset that he can t update his facebook by ...,0
3833,kenichan i dived many time for the ball manage...,0
3834,my whole body feel itchy and like it on fire,0
3835,nationwideclass no it s not behaving at all i ...,0


In [ ]:
# Replace 'is_depression' values of 0 with 5
df2['target'] = df2['is_depression'].replace(0, 5)
df2.drop('is_depression', axis=1, inplace=True)

In [ ]:
# Replace name 'clean_text' with 'text'
df2.rename(columns={'clean_text': 'text'}, inplace=True)
df2.head()

,text,target
3831,switchfoot http twitpic com y zl awww that s a...,5
3832,is upset that he can t update his facebook by ...,5
3833,kenichan i dived many time for the ball manage...,5
3834,my whole body feel itchy and like it on fire,5
3835,nationwideclass no it s not behaving at all i ...,5


In [ ]:
df2 = df2.sample(n=1200, random_state=42)  # Use random_state for reproducibility

In [ ]:
df2.shape

(1200, 2)

In [ ]:
# Merge the two dataframes
df_new = pd.concat([df1, df2], ignore_index=True)

# Display the first few rows of the merged dataframe
df_new.head()


,text,target
0,Welcome to /r/depression's check-in post - a p...,1
1,We understand that most people who reply immed...,1
2,Anyone else just miss physical touch? I crave ...,1
3,I’m just so ashamed. Everyone and everything f...,1
4,I really need a friend. I don't even have a si...,1


In [ ]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7157 entries, 0 to 7156
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    6807 non-null   object
 1   target  7157 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 112.0+ KB


In [ ]:
df_new.shape

(7157, 2)

In [ ]:
df_new['target'].value_counts()

1    1202
3    1201
5    1200
4    1188
2    1185
0    1181
Name: target, dtype: int64

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualize the distribution of the target variable
plt.figure(figsize=(8, 6))
sns.countplot(x='target', data=df_new)
plt.title('Distribution of Target Variable')
plt.xlabel('Target')
plt.ylabel('Count')
plt.show()

# You can also visualize other aspects of the dataset, such as:
# - Word cloud of the text data
# - Length distribution of text data
# - Correlation between features (if applicable)

# Example of visualizing text length
# Handle missing values in 'text' column before calculating length
df_new['text_length'] = df_new['text'].astype(str).apply(len)  # Convert to string to handle NaN
plt.figure(figsize=(8, 6))
sns.histplot(df_new['text_length'])
plt.title('Distribution of Text Length')
plt.xlabel('Text Length')
plt.ylabel('Count')
plt.show()

In [ ]:
import re

def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove mentions (e.g., @username)
    text = re.sub(r'@\w+', '', text)

    # Remove Reddit user mentions (e.g., /u/username)
    text = re.sub(r'/u/\w+', '', text)

    # Remove Reddit subreddit mentions (e.g., /r/subredditname)
    text = re.sub(r'/r/\w+', '', text)

    # Remove special characters and numbers
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply the cleaning function
df_new['cleaned_text'] = df_new['text'].apply(lambda x: clean_text(str(x)))

In [ ]:
df_new.head()

,text,target,cleaned_text
0,Welcome to /r/depression's check-in post - a p...,1,Welcome to s checkin post a place to take a mo...
1,We understand that most people who reply immed...,1,We understand that most people who reply immed...
2,Anyone else just miss physical touch? I crave ...,1,Anyone else just miss physical touch I crave i...
3,I’m just so ashamed. Everyone and everything f...,1,Im just so ashamed Everyone and everything fee...
4,I really need a friend. I don't even have a si...,1,I really need a friend I dont even have a sing...


In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return 'unknown'  # For cases where detection might fail

# Detect language
df_new['text_language'] = df_new['cleaned_text'].apply(lambda x: detect_language(str(x)))

In [ ]:
# Display the data with language detection
df_new.head()

,text,target,cleaned_text,text_language
0,Welcome to /r/depression's check-in post - a p...,1,Welcome to s checkin post a place to take a mo...,en
1,We understand that most people who reply immed...,1,We understand that most people who reply immed...,en
2,Anyone else just miss physical touch? I crave ...,1,Anyone else just miss physical touch I crave i...,en
3,I’m just so ashamed. Everyone and everything f...,1,Im just so ashamed Everyone and everything fee...,en
4,I really need a friend. I don't even have a si...,1,I really need a friend I dont even have a sing...,en


In [ ]:
# Remove rows where 'text_language' is english
df_new = df_new[df_new['text_language'] == 'en']

# Display the updated DataFrame
df_new.head()


,text,target,cleaned_text,text_language
0,Welcome to /r/depression's check-in post - a p...,1,Welcome to s checkin post a place to take a mo...,en
1,We understand that most people who reply immed...,1,We understand that most people who reply immed...,en
2,Anyone else just miss physical touch? I crave ...,1,Anyone else just miss physical touch I crave i...,en
3,I’m just so ashamed. Everyone and everything f...,1,Im just so ashamed Everyone and everything fee...,en
4,I really need a friend. I don't even have a si...,1,I really need a friend I dont even have a sing...,en


In [ ]:
df_new.shape

(6647, 4)

In [ ]:
df_new['target'].value_counts()

1    1190
4    1141
5    1120
2    1085
3    1058
0    1053
Name: target, dtype: int64

In [ ]:
!pip install transformers datasets torch

In [ ]:
!pip install datasets

In [ ]:
# Create an empty list to store the sampled dataframes
dfs_sampled = []

# Iterate through unique target values
for target_value in df_new['target'].unique():
  # Sample 220 rows for the current target value
  df_sampled = df_new[df_new['target'] == target_value].sample(n=220, random_state=42)
  dfs_sampled.append(df_sampled)

# Concatenate the sampled dataframes
df_new_test = pd.concat(dfs_sampled, ignore_index=True)

# Display the first few rows of the new dataframe
df_new_test.head()

,text,target,cleaned_text,text_language
0,This will probably go unseen but I need to wri...,1,This will probably go unseen but I need to wri...,en
1,"I'm not saying I have a plan to leave but man,...",1,Im not saying I have a plan to leave but man I...,en
2,It literally makes me go to the srart point wh...,1,It literally makes me go to the srart point wh...,en
3,i’ve had depression for five years now. and i’...,1,ive had depression for five years now and ill ...,en
4,I may cuss I’m sorry if I do that’s what the n...,1,I may cuss Im sorry if I do thats what the nsf...,en


In [ ]:
df_new_test.shape

(1320, 4)

In [ ]:
df_new_test['target'].value_counts()

1    220
0    220
2    220
3    220
4    220
5    220
Name: target, dtype: int64

In [ ]:
!pip install transformers

In [ ]:
!pip install transformers datasets

In [ ]:
!pip install --upgrade transformers datasets

In [ ]:
!pip install --upgrade transformers datasets accelerate

In [ ]:
!pip install --upgrade transformers

In [ ]:
pip install --upgrade tokenizers

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install torch datasets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!transformers-cli cache clear

usage: transformers-cli <command> [<args>]
Transformers CLI tool: error: argument {convert,download,env,run,serve,login,whoami,logout,repo,add-new-model-like,lfs-enable-largefiles,lfs-multipart-upload,pt-to-tf}: invalid choice: 'cache' (choose from 'convert', 'download', 'env', 'run', 'serve', 'login', 'whoami', 'logout', 'repo', 'add-new-model-like', 'lfs-enable-largefiles', 'lfs-multipart-upload', 'pt-to-tf')


In [ ]:
!pip uninstall transformers
!pip install transformers

^C
^C


In [ ]:
# Import necessary libraries
from transformers import BertTokenizer, BertForSequenceClassification
from transformers.training_args import TrainingArguments # Import TrainingArguments from transformers.training_args
from transformers.trainer import Trainer # Import Trainer from transformers.trainer
from transformers.data.data_collator import DataCollatorWithPadding # This is the changed line
from datasets import Dataset
import torch
import pandas as pd
import numpy as np

# Load dataset (Assuming df_new has 'cleaned_text' and 'target' columns)
df = df_new[['cleaned_text', 'target']]  # Ensure 'target' holds the labels 0: Stress, 1: Depression, etc.

# Train-validation split for df_new and df_new_test
train_texts, train_labels = df_new['cleaned_text'], df_new['target']
val_texts, val_labels = df_new_test['cleaned_text'], df_new_test['target']

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the texts for both training and validation sets in batches
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

# Convert data into Hugging Face Dataset format
train_dataset = Dataset.from_pandas(pd.DataFrame({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
}))

val_dataset = Dataset.from_pandas(pd.DataFrame({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
}))

# Load pretrained BERT model for sequence classification (6 output classes)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

# Use DataCollator for dynamic padding during training and evaluation
data_collator = DataCollatorWithPadding(tokenizer)

# Use class weights to handle imbalanced data (optional)
# Calculate class weights
class_counts = np.bincount(df['target'])
class_weights = torch.tensor([1.0 / count for count in class_counts]).float()

# Custom Trainer to override compute_loss method
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights.to(logits.device))
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Define Training Arguments (with optimizations for learning rate, batch size, and evaluation strategy)
training_args = TrainingArguments(
    output_dir='./results',               # Output directory for saved models and logs
    num_train_epochs=3,                   # Number of epochs
    per_device_train_batch_size=16,       # Train batch size (optimize for available memory)
    per_device_eval_batch_size=16,        # Evaluation batch size
    learning_rate=2e-5,                   # Optimized learning rate
    warmup_steps=500,                     # Warmup steps for learning rate scheduling
    weight_decay=0.01,                    # Regularization term to avoid overfitting
    logging_dir='./logs',                 # Directory for logging
    logging_steps=100,                    # Log every 100 steps
    eval_strategy="epoch",                # Evaluate after every epoch
    save_strategy="epoch",                # Save model after every epoch
    load_best_model_at_end=True,          # Load best model after training
    save_total_limit=2,                   # Save only the best 2 models to save space
)

# Initialize CustomTrainer (handles training and evaluation)
trainer = CustomTrainer(
    model=model,                           # BERT model with classification head
    args=training_args,                    # Training arguments
    train_dataset=train_dataset,           # Training dataset
    eval_dataset=val_dataset,              # Evaluation dataset
    data_collator=data_collator,           # Data collator for dynamic padding
)

# Train the model (fine-tune BERT on your depression classification task)
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained('./depression_classification_bert')
tokenizer.save_pretrained('./depression_classification_bert')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
!pip show transformers
!pip show tokenizers

Name: transformers
Version: 4.45.2
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: C:\Users\kochs\anaconda3\Lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
Name: tokenizers
Version: 0.20.0
Summary: 
Home-page: 
Author: Anthony MOI <m.anthony.moi@gmail.com>
Author-email: Nicolas Patry <patry.nicolas@protonmail.com>, Anthony Moi <anthony@huggingface.co>
License: 
Location: C:\Users\kochs\anaconda3\Lib\site-packages
Requires: huggingface-hub
Required-by: transformers


In [ ]:
pip install transformers --upgrade

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install tokenizers --upgrade

Note: you may need to restart the kernel to use updated packages.


In [ ]:
!transformers-cli cache clear

usage: transformers-cli <command> [<args>]
Transformers CLI tool: error: argument {convert,download,env,run,serve,login,whoami,logout,repo,add-new-model-like,lfs-enable-largefiles,lfs-multipart-upload,pt-to-tf}: invalid choice: 'cache' (choose from 'convert', 'download', 'env', 'run', 'serve', 'login', 'whoami', 'logout', 'repo', 'add-new-model-like', 'lfs-enable-largefiles', 'lfs-multipart-upload', 'pt-to-tf')


In [ ]:
!conda create -n transformers_env python=3.9
!conda activate transformers_env
!pip install transformers torch datasets

^C



EnvironmentNameNotFound: Could not find conda environment: transformers_env
You can list all discoverable environments with `conda info --envs`.




^C


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
import numpy as np
# Get predictions on the validation set
predictions = trainer.predict(val_dataset)

# Extract predicted labels and true labels
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

# Calculate precision, recall, F1-score, and accuracy
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate the confusion matrix
confusion = confusion_matrix(true_labels, predicted_labels)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print("\nConfusion Matrix:")
print(confusion)

NameError: name 'trainer' is not defined

In [ ]:
# Import necessary libraries
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the fine-tuned model and tokenizer
model = BertForSequenceClassification.from_pretrained('./depression_classification_bert')
tokenizer = BertTokenizer.from_pretrained('./depression_classification_bert')

# Set the model to evaluation mode
model.eval()

# Function to make predictions
def predict(texts):
    # Tokenize the input text
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

    # Move input tensors to the same device as the model
    input_ids = encodings['input_ids'].to(model.device)
    attention_mask = encodings['attention_mask'].to(model.device)

    with torch.no_grad():
        # Make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Get the predicted class indices
    predicted_class_indices = torch.argmax(logits, dim=1).cpu().numpy()

    return predicted_class_indices

# Example sentences to predict
sample_texts = [
    "I feel so happy and excited today!",
    "I can't seem to shake off this sadness.",
    "Everything feels overwhelming right now.",
    "I am just fine and content with my life."
]

# Get predictions for the sample texts
predictions = predict(sample_texts)

# Display the results
for text, pred in zip(sample_texts, predictions):
    print(f"Text: '{text}' | Predicted Class: {pred}")  # Adjust class labels based on your target encoding

# Save the fine-tuned model and tokenizer
model.save_pretrained('./depression_classification_bert')
tokenizer.save_pretrained('./depression_classification_bert')

In [ ]:
# Import necessary libraries
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load the fine-tuned model and tokenizer
model = BertForSequenceClassification.from_pretrained('./depression_classification_bert')
tokenizer = BertTokenizer.from_pretrained('./depression_classification_bert')

# Set the model to evaluation mode
model.eval()

# Function to make predictions
def predict(texts):
    # Tokenize the input text
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=128, return_tensors='pt')

    # Move input tensors to the same device as the model
    input_ids = encodings['input_ids'].to(model.device)
    attention_mask = encodings['attention_mask'].to(model.device)

    with torch.no_grad():
        # Make predictions
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Get the predicted class indices
    predicted_class_indices = torch.argmax(logits, dim=1).cpu().numpy()

    return predicted_class_indices

# Mapping of class indices to class labels (Adjust according to your classes)
class_labels = {
    0: "Stress",
    1: "Depression",
    2: "Bipolar Disorder",
    3: "Personality Disorder",
    4: "Anxiety",
    5: "Normal text"
}

# Function for user input and predictions
def main():
    while True:
        # Get user input
        user_input = input("Enter a sentence to classify (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break

        # Get predictions for the user input
        predictions = predict([user_input])

        # Display the results
        predicted_class = predictions[0]
        print(f"Text: '{user_input}' | Predicted Class: {class_labels[predicted_class]}")

# Run the main function to start the input loop
main()

Enter a sentence to classify (or type 'exit' to quit): exit
